# Visualizing The Fitting Parameters
We needed some sort of a metric to distinguish between the intensity v SDD curves for different model parameter combinations. The obvious choice was MSE. But MSE does not preserve any sense of the shape of the curve. Common metrics would be Frechet Distance(A person walking a dog, the trajectories of both are the curves, the max leash length), Hausdorf distance(The max. distance between any two points of both curves).  

None of those seemed useful for our case. Perhaps the best metric might be the fitting params. In this notebook, we explore how these fitting params change with different model parameters and if it truely is a good distance metric.  
(Beware, the data fitting is slow)

In [1]:
import seaborn as sns
from inverse_modelling_tfo.data import generate_data_loaders, normalize_zero_mean, get_interpolate_fit_params
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Load Data
data = pd.read_pickle(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/test_data.pkl')
print(len(data))    # Length sanity check
fitting_param_table = get_interpolate_fit_params(data, weights=[1, -3])
print(len(fitting_param_table))    # Length sanity check
# data = normalize_zero_mean(data)
data.head(10)

228000
228000


,SDD,Intensity,Wave Int,Uterus Thickness,Maternal Wall Thickness,Maternal Mu_a,Fetal Mu_a
0,10,3.607995e+06,2.0,6.0,8.0,0.005,0.05
1,15,4.764061e+05,2.0,6.0,8.0,0.005,0.05
2,19,8.680096e+04,2.0,6.0,8.0,0.005,0.05
3,24,1.666751e+04,2.0,6.0,8.0,0.005,0.05
4,29,3.162629e+03,2.0,6.0,8.0,0.005,0.05
5,34,6.383186e+02,2.0,6.0,8.0,0.005,0.05
6,38,1.153435e+02,2.0,6.0,8.0,0.005,0.05
7,43,2.318906e+01,2.0,6.0,8.0,0.005,0.05
8,48,4.419708e+00,2.0,6.0,8.0,0.005,0.05
9,53,8.987256e-01,2.0,6.0,8.0,0.005,0.05


In [2]:
parameter_names = ['Maternal Mu_a', 'Fetal Mu_a', 'Maternal Wall Thickness', 'Uterus Thickness']
for parameter in parameter_names:
    print(parameter, ' unique values ', pd.unique(data[parameter]))

Maternal Mu_a  unique values  [0.005  0.0055 0.006  0.0065 0.007  0.0075 0.008  0.0085 0.009  0.0095]
Fetal Mu_a  unique values  [0.05  0.055 0.06  0.065 0.07  0.075 0.08  0.085 0.09  0.095]
Maternal Wall Thickness  unique values  [ 8. 32. 26. 16. 30. 14. 18.  6. 36. 10. 12.  2. 38. 20.  4. 22. 28. 24.
 34.]
Uterus Thickness  unique values  [6. 5. 7.]


# Pivotting Data

In [10]:
fitting_param_table.head()

,Intensity,Wave Int,Uterus Thickness,Maternal Wall Thickness,Maternal Mu_a,Fetal Mu_a,alpha0,alpha1,alpha2,alpha3
0,3.607995e+06,2.0,6.0,8.0,0.0050,0.05,-0.220322,-1.392311,0.117994,21.448130
1,3.541480e+06,2.0,6.0,8.0,0.0055,0.05,-0.206672,-1.941621,1.106366,20.900324
2,3.476949e+06,2.0,6.0,8.0,0.0060,0.05,-0.192870,-2.495402,2.102762,20.348059
3,3.414321e+06,2.0,6.0,8.0,0.0065,0.05,-0.178904,-3.054028,3.107860,19.790950
4,3.353519e+06,2.0,6.0,8.0,0.0070,0.05,-0.164765,-3.617791,4.122183,19.228701


In [19]:
temp = pd.wide_to_long(fitting_param_table, i=['Intensity', 'Wave Int', 'Maternal Mu_a', 'Fetal Mu_a', 'Maternal Wall Thickness', 'Uterus Thickness'], stubnames='alpha', j='param_name')
temp.head()

alpha
Intensity    Wave Int Maternal Mu_a Fetal Mu_a Maternal Wall Thickness Uterus Thickness param_name           
3.607995e+06 2.0      0.0050        0.05       8.0                     6.0              0           -0.220322
                                                                                        1           -1.392311
                                                                                        2            0.117994
                                                                                        3           21.448130
3.541480e+06 2.0      0.0055        0.05       8.0                     6.0              0           -0.206672

# Changes in Fitting Params